# Exploring Parsing

Parsley vs Parsimonious

## Simple: Matching an Integer

In [1]:
INTEGER_TEST_CASES = {'12345': 12345,
                      '+100': 100,
                      '-4': -4}

### Parsimonious

In [2]:
from parsimonious import NodeVisitor
from parsimonious.grammar import Grammar

parsimonious_int_grammar = """
integer = ~"[+-]?[0-9]+"
"""

class IntEvaluator(NodeVisitor):
    def __init__(self, grammar, ctx, strict=True):
        self.grammar = Grammar(grammar)
        self._ctx = ctx
        self._strict = strict
        
    def visit_integer(self, node, children):
        return int(node.text)
    

grammar = IntEvaluator(parsimonious_int_grammar, {})
grammar.parse('12345')

12345

In [3]:
def test_integer_parsimonious():
    grammar = IntEvaluator(parsimonious_int_grammar, {})
    for key, val in INTEGER_TEST_CASES.items():
        print("checking {0!r:<15}".format(key), end='')
        assert grammar.parse(key) == val
        print("     ... ok")
        
test_integer_parsimonious()

checking '12345'             ... ok
checking '-4'                ... ok
checking '+100'              ... ok


### Parsley

In [4]:
import parsley

parsley_int_grammar = """
integer = <("+" | "-")? digit+>:d -> int(d)
"""

grammar = parsley.makeGrammar(parsley_int_grammar, {})
grammar('12345').integer()

12345

In [5]:
def test_integer_parsley():
    grammar = parsley.makeGrammar(parsley_int_grammar, {})
    for key, val in INTEGER_TEST_CASES.items():
        print("checking {0!r:<15}".format(key), end='')
        assert grammar(key).integer() == val
        print("     ... ok")
        
test_integer_parsley()

checking '12345'             ... ok
checking '-4'                ... ok
checking '+100'              ... ok


## Simple arithmetic parser

In [6]:
ARITHMETIC_TEST_CASES = {
    '43': 43,
    '-3': -3,
    '+2': +2,
    '1+2': ('add', 1, 2),
    '1 + 2': ('add', 1, 2),
    '1/-2': ('div', 1, -2),
    '44 *5': ('mul', 44, 5),
    '-564*    4': ('mul', -564, 4)
}

### Parsimonious

In [7]:
parsimonious_arith_grammar = r"""
add = (sub sp "+" sp add) / sub

sub = (mul sp "-" sp sub) / mul

mul = (div sp "*" sp mul) / div

div = (parens sp "/" sp div) / parens

parens = ("(" sp add sp ")") / integer

integer = ~"[+-]?[0-9]+"

sp = " "*
"""


class SimpleArithEvaluator(NodeVisitor):
    binary_expressions = ['add', 'sub', 'mul', 'div']
    
    def __init__(self, grammar, ctx, strict=True):
        self.grammar = Grammar(grammar)
        self._ctx = ctx
        self._strict = strict
    
    def _visit_binary(self, node, children):
        child = children[0]
        if not hasattr(child, '__len__'):
            return child
        if len(child) == 3:
            return child
        elif len(child) == 5:
            return (node.expr_name, child[0], child[4])
        
    visit_add = _visit_binary
    visit_sub = _visit_binary
    visit_mul = _visit_binary
    visit_div = _visit_binary
    
    def visit_parens(self, node, children):
        child = children[0]
        if not hasattr(child, '__len__'):
            return child
        elif len(child) == 5:
            return child[2]
    
    def visit_integer(self, node, children):
        return int(node.text)
    
    def generic_visit(self, node, children):
        return children
        
grammar = SimpleArithEvaluator(parsimonious_arith_grammar, {})
grammar.parse('4 * ((32 - 11) * -44 + 55) / 12')

('mul', 4, ('div', ('add', ('mul', ('sub', 32, 11), -44), 55), 12))

In [8]:
def test_arithmetic_parsimonious():
    grammar = SimpleArithEvaluator(parsimonious_arith_grammar, {})
    for key, val in ARITHMETIC_TEST_CASES.items():
        print("checking {0!r:<15}".format(key), end='')
        assert grammar.parse(key) == val
        print("     ... ok")
        
test_arithmetic_parsimonious()

checking '1/-2'              ... ok
checking '1 + 2'             ... ok
checking '-3'                ... ok
checking '43'                ... ok
checking '+2'                ... ok
checking '44 *5'             ... ok
checking '-564*    4'        ... ok
checking '1+2'               ... ok


In [9]:
%timeit grammar.parse('(4 * ((32 - 11) * -44 + 55) / 12) * (60 - 32 + 581)')

1000 loops, best of 3: 989 µs per loop


### Parsley

In [10]:
parsley_arith_grammar = """
parse = add

add = sub:left sp "+" sp add:right -> ('add', left, right)
      | sub:child                  -> child

sub = mul:left sp "-" sp sub:right -> ('sub', left, right)
      | mul:child                  -> child

mul = div:left sp "*" sp mul:right -> ('mul', left, right)
      | div:child                  -> child

div = parens:left sp "/" sp div:right -> ('div', left, right)
      | parens:child                  -> child
      
parens = "(" sp add:child sp ")"  -> child
          | integer:child          -> child

sp = ' '*
integer = <("+" | "-")?digit+>:d -> int(d)
"""

grammar = parsley.makeGrammar(parsley_arith_grammar, {})
grammar('4 * ((32 - 11) * -44 + 55) / 12').parse()

('mul', 4, ('div', ('add', ('mul', ('sub', 32, 11), -44), 55), 12))

In [11]:
def test_arithmetic_parsley():
    grammar = parsley.makeGrammar(parsley_arith_grammar, {})
    for key, val in ARITHMETIC_TEST_CASES.items():
        print("checking {0!r:<15}".format(key), end='')
        assert grammar(key).parse() == val
        print("     ... ok")
        
test_arithmetic_parsley()

checking '1/-2'              ... ok
checking '1 + 2'             ... ok
checking '-3'                ... ok
checking '43'                ... ok
checking '+2'                ... ok
checking '44 *5'             ... ok
checking '-564*    4'        ... ok
checking '1+2'               ... ok


In [12]:
%timeit grammar('(4 * ((32 - 11) * -44 + 55) / 12) * (60 - 32 + 581)').parse()

100 loops, best of 3: 3.35 ms per loop


## Matching numbers

Let's build a parser that matches numbers... int, binary, octal, hex, and floats

In [13]:
NUMBER_TEST_CASES = {
    # decimal integers
    '12345': 12345,
    '+100': 100,
    '-42': -42,
    
    # binary integers
    '0b11': 0b11,
    '-0b101': -0b101,
    '0B01': 0b01,
    '-0B001': -0b001,
    
    # octal integers
    '012': 0o12,
    '-0132': -0o132,
    '0o12': 0o12,
    '-0o132': -0o132,
    '0O12': 0o12,
    '-0O132': -0o132,
    
    # hexadecimal integers
    '0x42': 0x42,
    '-0xFA': -0xFA,
    '0X0F0': 0x0F0,
    '-0X5DEF': -0x5DEF,
    '0x42': 0x42,
    '-0xFA': -0xfA,
    '0X0F0': 0x0f0,
    '-0X5DEF': -0x5DeF,
    
    # floating point
    '11.': 11.0,
    '0E5': 0.0,
    '1.1': 1.1,
    '0.1': 0.1,
    '+.1': 0.1,
    '.11': 0.11,
    '1E4': 1E4,
    '1.2E-3': 1.2E-3,
    '-1E-4': -1E-4,
    '-1.2E-3': -1.2E-3,
    '12e4': 12E4,
    '1.22e-3': 1.22E-3,
    '-12e-4': -12E-4,
    '-1.22e-3': -1.22E-3,
}

### Parsimonious

In [14]:
parsimonious_number_grammar = r"""
number = hexadecimal / binary / octal / float / integer

float = exponent_float / decimal_float

exponent_float = (decimal_float / integer_part) ("E" / "e") integer_part

decimal_float = ~r"[+-]?([0-9]+[.][0-9]*|[.][0-9]+)"

# integer part of floats can start with zero
integer_part = ~r"[+-]?[0-9]+"

hexadecimal = ~r"[+-]?0[Xx][0-9A-Fa-f]+"

binary = ~r"[+-]?0[Bb][0-1]+"

octal = ~r"[+-]?0[Oo]?[0-7]+"

# integers cannot start with 0 as that indicates octal
integer = ~r"[+-]?[1-9][0-9]*"
"""

class NumberEvaluator(NodeVisitor):
    def __init__(self, grammar, ctx, strict=True):
        self.grammar = Grammar(grammar)
        self._ctx = ctx
        self._strict = strict
        
    def visit_number(self, node, children):
        return children[0]
        
    def visit_float(self, node, children):
        return float(node.text)
    
    def visit_hexadecimal(self, node, children):
        return int(node.text, 16)
    
    def visit_binary(self, node, children):
        return int(node.text, 2)
    
    def visit_octal(self, node, children):
        return int(node.text, 8)
    
    def visit_integer(self, node, children):
        return int(node.text)
    
    def generic_visit(self, node, children):
        return children
    

grammar = NumberEvaluator(parsimonious_number_grammar, {})
grammar.parse('-11.0')

-11.0

In [15]:
def test_number_parsimonious():
    grammar = NumberEvaluator(parsimonious_number_grammar, {})
    for key, val in NUMBER_TEST_CASES.items():
        print("checking {0!r:<15}".format(key), end='')
        assert grammar.parse(key) == val
        print("     ... ok")
        
test_number_parsimonious()

checking '12e4'              ... ok
checking '-1.22e-3'          ... ok
checking '-0o132'            ... ok
checking '-0b101'            ... ok
checking '0B01'              ... ok
checking '12345'             ... ok
checking '0E5'               ... ok
checking '0O12'              ... ok
checking '-1.2E-3'           ... ok
checking '+.1'               ... ok
checking '-1E-4'             ... ok
checking '.11'               ... ok
checking '0x42'              ... ok
checking '-0B001'            ... ok
checking '0o12'              ... ok
checking '0.1'               ... ok
checking '-0xFA'             ... ok
checking '012'               ... ok
checking '0b11'              ... ok
checking '0X0F0'             ... ok
checking '1.2E-3'            ... ok
checking '-0X5DEF'           ... ok
checking '1.1'               ... ok
checking '1.22e-3'           ... ok
checking '11.'               ... ok
checking '-0O132'            ... ok
checking '+100'              ... ok
checking '-12e-4'           

### Parsley

In [16]:
parsley_number_grammar = """
number = hexadecimal | octal | binary | float | integer

binary = <sign "0" ("B" | "b") digit0_1+>:d -> int(d, 2)

octal = <sign "0" ("O" | "o")? digit0_7+>:d -> int(d, 8)

hexadecimal = <sign "0" ("X" | "x") digit_hex+>:d -> int(d, 16)

float = exponent_float | decimal_float

exponent_float = <sign (decimal_float | digit+) ("E" | "e") sign digit+>:d -> float(d)

decimal_float = <sign digit+ "." digit*>:d -> float(d)
                | <sign "." digit+>:d      -> float(d)

# integers cannot start with 0 as that indicates octal
integer = <sign digit1_9 digit*>:d -> int(d)

# building blocks
sign = <("+" | "-")?>:s -> s
digit = :x ?(x in '0123456789') -> x
digit1_9 = :x ?(x in '123456789') -> x
digit0_7 = :x ?(x in '01234567') -> x
digit0_1 = :x ?(x in '01') -> x
digit_hex = :x ?(x in '0123456789abcdefABCDEF') -> x
"""

grammar = parsley.makeGrammar(parsley_number_grammar, {})
grammar('-400.0E5').number()

-40000000.0

In [17]:
def test_number_parsley():
    grammar = parsley.makeGrammar(parsley_number_grammar, {})
    for key, val in NUMBER_TEST_CASES.items():
        print("checking {0!r:<15}".format(key), end='')
        assert grammar(key).number() == val
        print("     ... ok")
        
test_number_parsley()

checking '12e4'              ... ok
checking '-1.22e-3'          ... ok
checking '-0o132'            ... ok
checking '-0b101'            ... ok
checking '0B01'              ... ok
checking '12345'             ... ok
checking '0E5'               ... ok
checking '0O12'              ... ok
checking '-1.2E-3'           ... ok
checking '+.1'               ... ok
checking '-1E-4'             ... ok
checking '.11'               ... ok
checking '0x42'              ... ok
checking '-0B001'            ... ok
checking '0o12'              ... ok
checking '0.1'               ... ok
checking '-0xFA'             ... ok
checking '012'               ... ok
checking '0b11'              ... ok
checking '0X0F0'             ... ok
checking '1.2E-3'            ... ok
checking '-0X5DEF'           ... ok
checking '1.1'               ... ok
checking '1.22e-3'           ... ok
checking '11.'               ... ok
checking '-0O132'            ... ok
checking '+100'              ... ok
checking '-12e-4'           

# Scratch space

In [ ]:
parsimonious_literal_grammar = r"""
literal = string / float / integer

string = ~r"([\'\"])([^\1]*)\1"

float = exponent_float / decimal_float
exponent_float = (decimal_float / (sign digits)) ~"[Ee]" sign digits
decimal_float = (sign digits "." digits) / (sign digits ".") / ("." digits)

integer = sign digits

sign = ~"[+-]?"
digits = ~"[0-9]+"
"""

class LiteralEvaluator(NodeVisitor):
    def __init__(self, grammar, ctx, strict=True):
        self.grammar = Grammar(grammar)
        self._ctx = ctx
        self._strict = strict
    
    def visit_string(self, node, children):
        return node.match.groups()[1]
    
    def visit_float(self, node, children):
        return float(node.full_text)
    
    def visit_integer(self, node, children):
        return int(node.full_text)
    
    def visit_literal(self, node, children):
        return children[0]
    
    def generic_visit(self, node, children):
        return children

grammar = LiteralEvaluator(parsimonious_literal_grammar, {})
grammar.parse('1.4E6')

In [ ]:
LITERAL_TEST_CASES = {'100': 100,
                      '2.5': 2.5,
                      '.5': 0.5,
                      '5.': 5.0,
                      '1E6': 1E6,
                      '1e6': 1e6,
                      '1.4E7': 1.4E7,
                      '"hello there agent #7"': 'hello there agent #7',
                      "'hello 45.9/!'": 'hello 45.9/!'}

def test_literal_evaluator():
    parser = LiteralEvaluator(parsimonious_literal_grammar, {})

    for key, val in LITERAL_TEST_CASES.items():
        result = parser.parse(key)
        assert type(result) == type(val)
        assert result == val
        
test_literal_evaluator()

In [ ]:
grammar = Grammar(r"""
expr
expr = binop / parens / func / identifier / sequence / literal

parens = "(" space expr space ")"


# Arithmetic Operators

binop = muldiv / addsub

addsub = add / sub

muldiv = mul / div

add = expr space "+" space expr

sub = expr space "-" space expr

mul = expr space "*" space expr

div = expr space "/" space expr


# Functions and Identifiers

func = identifier space "(" seq ")"

identifier = ~"[_a-zA-Z][_a-zA-Z0-9]*"


# Lists and mappings

sequence = list / tuple / set / mapping

list = "[" seq "]"

tuple = "(" seq ")"

set = "{" seq "}"

mapping = "{" mappingseq "}"

seq = space (seqitem space "," space)* (seqitem space)?

seqitem = expr

mappingseq = space (mappingitem space "," space)* (mappingitem space)?

mappingitem = identifier space ":" space expr


# Literal objects

literal = string / float / integer

string = ~"\"[^\"]*\"|\'[^\']*\'"

number = float / integer

float = (integer "." digits) / (integer ".") / ("." digits) / (integer ~"[Ee]" integer)

integer = "-"? digits


# Building blocks

space = " "*
digits = ~"[0-9]+"
""")

#g = grammar.parse('[foo( -2e-6, blah ), 5.0]')
g = grammar.parse('{hello: blah, foo: "bar"} + 4')
print(g.full_text)
print(g)

In [ ]:
# precedence

# comma/sequence
# inline conditional ? :
# logical or
# logical and
# equality/inequality/strict equality/strict inequality
# greater/less than (or equal to)
# bit-shifts << >> >>>
# addition/subtraction
# multiplication/division/mod/exponentiation
# binary/logical not; unary +/-
# function call
# member access via . or []
# parens

'11.'   -> float
'1.1'  -> float
'.11'   -> float
'11'   -> decimal
'0b11' -> binary
'011'  -> octal
'0x11' -> hexadecimal

str has "" or ''

In [ ]:
import parsley
numbers = parsley.makeGrammar("""
number = float | integer
integer = <digit+>:d -> int(d)
float = "." <digit+>:B              -> float('.' + B)
        | <digit+>:A "." <digit*>:B -> float(A + '.' + B)
        | <digit+>:val1 "."         -> float(A + '.')
""", {})

numbers('4.').number()

In [ ]:
import parsley

def calculate(start, pairs):
    result = start
    for op, value in pairs:
        if op == '+':
            result += value
        elif op == '-':
            result -= value
        elif op == '*':
            result *= value
        elif op == '/':
            result /= value
    return result

grammar = parsley.makeGrammar("""
number = <digit+>:ds -> int(ds)
parens = '(' ws expr:e ws ')' -> e
value = number | parens
ws = ' '*
add = '+' ws expr2:n -> ('+', n)
sub = '-' ws expr2:n -> ('-', n)
mul = '*' ws value:n -> ('*', n)
div = '/' ws value:n -> ('/', n)

addsub = ws (add | sub)
muldiv = ws (mul | div)

expr = expr2:left addsub*:right -> calculate(left, right)
expr2 = value:left muldiv*:right -> calculate(left, right)
""", {"calculate": calculate})

g = grammar("4 * (5 + 6) + 1")

In [ ]:
grammar??

In [ ]:
grammar = parsley.makeGrammar("""
integer = <digit+>:val -> int(val)
float = "." <digit+>:val2                 -> float('.' + val2)
        | <digit+>:val1 "." <digit*>:val2 -> float(val1 + '.' + val2)
        | <digit+>:val1 "."               -> float(val1)
number = (float | integer)

identifier = <(letter | '_')>:first <(letter | digit | '_')*>:rest -> first + rest
ws = " "*

parens = "(" ws value:v ws ")" -> v

func = identifier:funcname ws "(" ws value:arg ws ")" -> (funcname, arg)

value = func | parens | identifier | number
""", {})

grammar('foo(x)').value()